<a href="https://colab.research.google.com/github/vsoumyareddy/CODSOFT/blob/main/sms_spam_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sms-spam-collection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F483%2F982%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240419%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240419T155205Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2143886a62fd72272e251bb0d4ec3a4cdb1b01f883cb616af576008522e8189544ffe2ff696feda183467d58218e27128938204e4404194e69f2d5e29f74fdb67d7ed181ab621178a09c8176c9e404628976f971b5ab2f0c061919d3a4a29ffbea34615d461e42cbacdb44154183c5c53787abf7a8f19f75b7e1b8e128d804f53ecfe605d97f7a0f44434b48caf0c5a7d458d220ab2a069c01cfe1e4ebbffbbbdbef8df378fb96c9b6eeefe0712e5716c16f9627c2b3fbe0748e423150cbd8ca4ea997f49e7f0985af78388c9abdb89d3201b0cc8bb476f988d6deff6547eae4ec8a7ed805d357940bf8c742621868e7f0157c84335388631649ee93c760890e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score,
                             recall_score, f1_score)

In [ ]:
df=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv",encoding="ISO-8859-1")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [ ]:
df.head()

In [ ]:
# Checking for null values
df.isna().sum()

In [ ]:
# Checking and droping Duplicate values
df.duplicated().sum()
df=df.drop_duplicates()

In [ ]:
# Converting Target column into numerical 0 and 1
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
df['v1']=encoder.fit_transform(df['v1'])

In [ ]:
df.head()

In [ ]:
df['v1'].value_counts()

In [ ]:
# Check the Target value using pie chart
plt.pie(df['v1'].value_counts(), labels=['Ham','Spam'],autopct='%.2f')
plt.show()

DATA PREPROCESSING

In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
string.punctuation
from nltk.stem.porter import PorterStemmer

In [ ]:
def process_text(text):
  text=text.lower()

#tokenization
  text=nltk.word_tokenize(text)

# removing stopwords an punctuation
  x=[]
  for i in text:
      if i not in stopwords.words('english') and i not in string.punctuation:
        x.append(i)

  text=x[:]
  x.clear( )

  ps=PorterStemmer()
  for i in text:
    x.append(ps.stem(i))

  return " ".join(x)


In [ ]:
df['Transformed_text']=df['v2'].apply(process_text)

In [ ]:
df.head()


TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_features=3000)

In [ ]:
# Fit and transform the 'Transformed_text' column
X = tfidf.fit_transform(df['Transformed_text'])


print("Shape of X:", X.shape)

In [ ]:
X.shape

In [ ]:
y = df['v1'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X.toarray(),y,test_size=0.2,random_state=2)

LOGISTIC REGRESSION

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
accuracy_score(y_train, lr.predict(x_train))

In [ ]:
accuracy_score(y_test, lr.predict(x_test))

In [ ]:
table_1={}
table_1["LOGISTIC REGRESSION"] = (accuracy_score(y_test, lr.predict(x_test)),
                               precision_score(y_test, lr.predict(x_test)),
                               recall_score(y_test, lr.predict(x_test)),
                               f1_score(y_test, lr.predict(x_test)))

SVM

In [ ]:
svm= SVC()
svm.fit(x_train, y_train)
accuracy_score(y_train, svm.predict(x_train))

In [ ]:
accuracy_score(y_test, svm.predict(x_test))

In [ ]:
table_1["SVM"] = (accuracy_score(y_test, svm.predict(x_test)),
                               precision_score(y_test, svm.predict(x_test)),
                               recall_score(y_test, svm.predict(x_test)),
                               f1_score(y_test, svm.predict(x_test)))

NAVIE BAYES

In [ ]:
nb = GaussianNB()
nb.fit(x_train, y_train)
accuracy_score(y_train, nb.predict(x_train))

In [ ]:
accuracy_score(y_test, nb.predict(x_test))

In [ ]:
table_1["NAVIE BAYES"] = (accuracy_score(y_test, nb.predict(x_test)),
                               precision_score(y_test, nb.predict(x_test)),
                               recall_score(y_test, nb.predict(x_test)),
                               f1_score(y_test, nb.predict(x_test)))

In [ ]:
pd.DataFrame(table_1, index=["accuracy", "precision", "recall", "f1score"])